In [1]:
import numpy as np
import sympy as sym
from sympy import Function, hessian, pprint, Matrix, simplify
import math
from scipy.spatial.transform import Rotation as R
from sympy.parsing.sympy_parser import parse_expr
import time

sym.init_printing()

def angles_to_rad (q):
    for i in range (q.size):
        q[i] = np.deg2rad(q[i])

def get_uniform_joints_angles_within_limits():
    # angle limits for each robot joint in deg
    q_min = np.array ([-20,-50,-85,-145,-105,-195], dtype = 'float')
    q_max = np.array([95,55,40,145,105,195], dtype = 'float')
    # tranform angles to radians
    angles_to_rad(q_min)
    angles_to_rad(q_max)
    max_delta = q_max-q_min
    array_of_q = q_min.copy()
    q_current = q_min.copy()
    investigated_poses = 10
    for i in range (investigated_poses):
        q_current += (1/investigated_poses) * max_delta
        array_of_q = np.vstack((array_of_q, q_current))
    print (array_of_q.size)
    return array_of_q

def print_pose(pose):
    print("position of robot flange:", pose[0:3,3])
    print ("x vector:" , pose[0:3:,0])
    print ("y vector:" , pose[0:3:,1])
    print ("z vector:" , pose[0:3:,2]) 

# 24 nominal parameters of kinematics
def get_nominal_DH_kinematic_parameters():
    kinematic_parameters =                                  np.array([525,150,0,-90], dtype = 'float')
    kinematic_parameters = np.vstack((kinematic_parameters, np.array([0,790,-90,180])))
    kinematic_parameters = np.vstack((kinematic_parameters, np.array([0,250,0,-90])))    
    kinematic_parameters = np.vstack((kinematic_parameters, np.array([-1040,0,0,90])))
    kinematic_parameters = np.vstack((kinematic_parameters, np.array([0,0,0,-90])))
    kinematic_parameters = np.vstack((kinematic_parameters, np.array([-100,0,0,0])))
    return kinematic_parameters

print("symbolic calculations started",time.strftime("%H:%M:%S", time.localtime()))
def generate_DH_error_symbols(n):
    generated_symbols = []
    for i in range (n):
        generated_symbols.append(sym.symbols('δd'+ str(i)))
        generated_symbols.append(sym.symbols('δa'+ str(i)))
        generated_symbols.append(sym.symbols('δθ'+ str(i)))
        generated_symbols.append(sym.symbols('δα'+ str(i)))
    return generated_symbols

d, a,θ,α = sym.symbols('d a θ α')
q_symbols = sym.symbols('q:6')
error_symbols = generate_DH_error_symbols(6)   

def get_symbolic_matrix_with_error (δd,δa,δθ,δα, kinematics_params, q):
    [d0,a0,θ0,α0] = kinematics_params
    θ0 = np.deg2rad(θ0)
    α0 = np.deg2rad(α0)
    d = d0 + δd
    a = a0 +δa
    θ = θ0 + q + δθ
    α = α0 + δα
    
    dh = sym.eye(4)
    dh[0,0] = sym.cos(θ)
    dh[0,1] = -sym.sin(θ) * sym.cos(α)
    dh[0,2] = sym.sin(θ) * sym.sin(α)
    dh[0,3] = a * sym.cos(θ)
    dh[1,0] = sym.sin(θ)
    dh[1,1] = sym.cos(θ)*sym.cos(α)
    dh[1,2] = -sym.cos(θ)* sym.sin(α)
    dh[1,3] = a *sym.sin(θ)
    dh[2,1] = sym.sin(α)
    dh[2,2] = sym.cos(α)
    dh[2,3] = d
    return dh

def get_symbolic_kinematics_matrix():
    nominal_kinematics = get_nominal_DH_kinematic_parameters()
    result = sym.eye(4)
    for i in range(6):
        result = result @ get_symbolic_matrix_with_error(
            *error_symbols[4*i:4*(i+1)],nominal_kinematics[i], q_symbols[i])
    return result

symbolic_matrix_with_error = get_symbolic_kinematics_matrix()
calc_pose_with_error = sym.lambdify([q_symbols,error_symbols], 
                                           symbolic_matrix_with_error, "numpy") 

def get_symbolic_jacobian():
    position_function = symbolic_matrix_with_error[0:3,3]
    jacobian = position_function.jacobian(error_symbols)
    return jacobian

symbolic_jacobian = get_symbolic_jacobian()
calc_jacobian_for_angle = sym.lambdify([q_symbols,error_symbols], symbolic_jacobian, "numpy") 

print("symbolic calculations completed", time.strftime("%H:%M:%S", time.localtime()))

symbolic calculations started 15:13:02
symbolic calculations completed 15:13:16


In [3]:
# builds jacobian matrix for observations in q_values 
def form_full_jacobian(q_values,error):
    full_jacobian = np.empty((0,error.size), float)
    for q in q_values:
        full_jacobian = np.append(full_jacobian, calc_jacobian_for_angle(q,error), axis=0)
    return full_jacobian

print("numeric calculations started", time.strftime("%H:%M:%S", time.localtime()))
zero_error = np.zeros(24, dtype = 'float')
investigated_joint_angles = get_uniform_joints_angles_within_limits()
full_jacobian = form_full_jacobian(investigated_joint_angles,zero_error)

q_test = np.array ([-10,10,15,17,10,15], dtype = 'float')
angles_to_rad(q_test)

print("pose with error", calc_pose_with_error(q_test, zero_error)[0:3,3])
print("numeric calculations completed", time.strftime("%H:%M:%S", time.localtime()))


numeric calculations started 15:17:46
66
pose with error [1377.73573789 -237.77668088 1667.81480373]
numeric calculations completed 15:17:46


In [ ]:
#deprecated code to calculate pose for nominal kinematics and kinematics with e

def calculate_jacobian_at_angles(error_parameters,q_current):
    calc_jacobian_column()
    symbolic_matrix = get_symbolic_kinematics_matrix()    
    position_function = symbolic_matrix[0:3,3]
    jacobian = position_function.jacobian(error_symbols)
    calc_jacobian = sym.lambdify(error_symbols, jacobian, "numpy") 
    return calc_jacobian(*error_parameters)

def get_kinematics_model_with_noise(kinematic_parameters):
    for param in np.nditer(kinematic_parameters, op_flags=['readwrite']):
        param[...] = param + np.random.normal(0,0.15)
    return kinematic_parameters

# symbolic DH matrix
def get_symbolic_dh_matrix (d,a,θ,α):
    dh = sym.eye(4)
    dh[0,0] = sym.cos(θ)
    dh[0,1] = -sym.sin(θ) * sym.cos(α)
    dh[0,2] = sym.sin(θ) * sym.sin(α)
    dh[0,3] = a * sym.cos(θ)
    dh[1,0] = sym.sin(θ)
    dh[1,1] = sym.cos(θ)*sym.cos(α)
    dh[1,2] = -sym.cos(θ)* sym.sin(α)
    dh[1,3] = a *sym.sin(θ)
    dh[2,1] = sym.sin(α)
    dh[2,2] = sym.cos(α)
    dh[2,3] = d

    return dh

dh_matrix_without_error = get_symbolic_dh_matrix(d,a,θ,α)
calc_dh_matrix = sym.lambdify([d, a,θ,α], dh_matrix_without_error, "numpy") 

# calculate DH matrix with given parameters and angle θ
def get_matrix_for_angle (d,a,θ0,α, θ):
    matrix = calc_dh_matrix(d,a,np.deg2rad(θ0 +θ),np.deg2rad(α))
    return matrix
    
def get_pose (θ,kinematics):
    resulting_transform = sym.eye(4)
    for i in range(6):
        resulting_transform = resulting_transform @ get_matrix_for_angle(*kinematics[i], θ[i])
    return resulting_transform

q_test = np.array ([10,10,10,17,10,10], dtype = 'float')
nominal_pose = get_pose(q_test,get_nominal_DH_kinematic_parameters())

print("nominal pose",nominal_pose[0:3,3])